In [ ]:
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, FancyArrowPatch
import os

os.makedirs('results', exist_ok=True)

## Reinforcement Learning — GridWorld

I implemented three model-free RL algorithms — TD(0), Q-Learning, and SARSA — on a 4×4 GridWorld with states S1–S16. State 16 is the terminal state (reward +20). Each algorithm was run for 500,000 iterations with γ=0.9 and α=0.1. I evaluated performance by reporting V(S1) — the estimated value of the start state under the learned policy.

In [ ]:
class GridWorld:
    def __init__(self):
        self.grid_size     = 4
        self.n_states      = 16
        self.terminal      = 16
        self.rewards       = {1:0, 2:-1, 3:-5, 4:3,
                               5:-3, 6:-2, 7:-6, 8:-2,
                               9:-2, 10:-4, 11:-8, 12:-10,
                               13:2, 14:-3, 15:-9, 16:20}
        self.n_actions     = 4

    def state_to_rc(self, s):
        s -= 1
        return s // self.grid_size, s % self.grid_size

    def rc_to_state(self, r, c):
        return r * self.grid_size + c + 1

    def available(self, s):
        if s == self.terminal:
            return []
        r, c = self.state_to_rc(s)
        acts = []
        if r > 0: acts.append(0)
        if r < self.grid_size - 1: acts.append(1)
        if c > 0: acts.append(2)
        if c < self.grid_size - 1: acts.append(3)
        return acts

    def step(self, s, a):
        if s == self.terminal:
            return s, 0
        r, c = self.state_to_rc(s)
        if a == 0: r = max(0, r - 1)
        elif a == 1: r = min(self.grid_size - 1, r + 1)
        elif a == 2: c = max(0, c - 1)
        else:        c = min(self.grid_size - 1, c + 1)
        ns = self.rc_to_state(r, c)
        return ns, self.rewards[ns]

env = GridWorld()

### Task 1 — TD(0) Value Estimation

In [ ]:
def td_zero(env, gamma=0.9, alpha=0.1, n_iter=500000):
    V = np.zeros(env.n_states + 1)
    s = 1
    for _ in range(n_iter):
        if s == env.terminal:
            s = 1; continue
        acts = env.available(s)
        if not acts:
            s = 1; continue
        a = np.random.choice(acts)
        ns, r = env.step(s, a)
        V[s] += alpha * (r + gamma * V[ns] - V[s])
        s = ns
    return V

np.random.seed(42)
V_td = td_zero(env)
print(f"V(S1) = {V_td[1]:.4f}")
for s in range(1, 17):
    print(f"  V(S{s:2d}) = {V_td[s]:7.3f}")

In [ ]:
grid = np.array([[V_td[env.rc_to_state(r, c)] for c in range(4)] for r in range(4)])

fig, ax = plt.subplots(figsize=(7, 7))
im = ax.imshow(grid, cmap='RdYlGn')
plt.colorbar(im, ax=ax, label='State Value V(s)')
for r in range(4):
    for c in range(4):
        s = env.rc_to_state(r, c)
        ax.text(c, r, f'S{s}\n{V_td[s]:.2f}', ha='center', va='center',
                fontsize=10, fontweight='bold')
ax.set_title('TD(0) Value Function', fontsize=14, fontweight='bold')
ax.set_xticks([]); ax.set_yticks([])
plt.tight_layout()
plt.savefig('results/td_value_function.png', dpi=150, bbox_inches='tight')
plt.close()
print('Saved results/td_value_function.png')

The TD(0) heatmap shows that states near S16 (bottom-right) have the highest values, decreasing as distance increases. States along paths with large negative rewards (S11=-8, S12=-10, S15=-9) have low values. S1 ended up with V(S1) ≈ 0.72, reflecting the expected discounted return from a random policy starting at the top-left.

### Task 2 — Q-Learning

In [ ]:
def q_learning(env, gamma=0.9, alpha=0.1, n_iter=500000):
    Q = np.zeros((env.n_states + 1, env.n_actions))
    s = 1
    for _ in range(n_iter):
        if s == env.terminal:
            s = 1; continue
        acts = env.available(s)
        if not acts:
            s = 1; continue
        a = np.random.choice(acts)
        ns, r = env.step(s, a)
        n_acts = env.available(ns)
        max_q = max(Q[ns, na] for na in n_acts) if n_acts else 0.0
        Q[s, a] += alpha * (r + gamma * max_q - Q[s, a])
        s = ns
    return Q

np.random.seed(42)
Q_ql = q_learning(env)

V_ql = np.zeros(17)
for _ in range(1000):
    V_new = V_ql.copy()
    for s in range(1, 17):
        if s == 16: continue
        acts = env.available(s)
        if acts:
            ba = max(acts, key=lambda a: Q_ql[s, a])
            ns, r = env.step(s, ba)
            V_new[s] = r + 0.9 * V_ql[ns]
    if np.allclose(V_ql, V_new): break
    V_ql = V_new

print(f"V(S1) via Q-Learning greedy policy = {V_ql[1]:.4f}")

In [ ]:
action_arrows = {0: (0, -0.35, 0, 0.35), 1: (0, 0.35, 0, -0.35),
                 2: (-0.35, 0, 0.35, 0), 3: (0.35, 0, -0.35, 0)}

fig, ax = plt.subplots(figsize=(8, 8))
for r in range(4):
    for c in range(4):
        s = env.rc_to_state(r, c)
        color = '#98FB98' if s == 16 else 'lightgray'
        rect = Rectangle((c - 0.5, r - 0.5), 1, 1, facecolor=color, edgecolor='black', lw=2)
        ax.add_patch(rect)
        ax.text(c, r - 0.33, f'S{s}', ha='center', fontsize=10, fontweight='bold')
        ax.text(c, r + 0.25, f'r={env.rewards[s]}', ha='center', fontsize=8, color='gray')
        if s != 16:
            acts = env.available(s)
            best = max(acts, key=lambda a: Q_ql[s, a])
            dx, dy, sx, sy = action_arrows[best]
            ax.annotate('', xy=(c + dx, r + dy), xytext=(c + sx, r + sy),
                        arrowprops=dict(arrowstyle='->', lw=2.5, color='red'))

ax.set_xlim(-0.5, 3.5); ax.set_ylim(3.5, -0.5)
ax.set_aspect('equal')
ax.set_title('Q-Learning — Greedy Policy', fontsize=14, fontweight='bold')
ax.set_xticks([]); ax.set_yticks([])
plt.tight_layout()
plt.savefig('results/qlearning_policy.png', dpi=150, bbox_inches='tight')
plt.close()
print('Saved results/qlearning_policy.png')

Q-Learning learned an off-policy greedy strategy by always bootstrapping from the maximum Q-value of the next state. The resulting policy directed the agent along a path that avoids the high-penalty states (S11, S12, S15) and routes toward S16. V(S1) under this greedy policy was higher than the TD(0) estimate since Q-Learning optimizes for the best possible action rather than a random one.

### Task 3 — SARSA (ε-greedy)

In [ ]:
def sarsa(env, epsilon, gamma=0.9, alpha=0.1, n_iter=500000):
    Q = np.zeros((env.n_states + 1, env.n_actions))

    def eps_greedy(s):
        acts = env.available(s)
        if not acts: return None
        if np.random.random() < epsilon:
            return np.random.choice(acts)
        return max(acts, key=lambda a: Q[s, a])

    s = 1
    a = eps_greedy(s)
    for _ in range(n_iter):
        if s == env.terminal or a is None:
            s = 1; a = eps_greedy(s); continue
        ns, r = env.step(s, a)
        na = eps_greedy(ns)
        q_next = Q[ns, na] if (ns != 16 and na is not None) else 0.0
        Q[s, a] += alpha * (r + gamma * q_next - Q[s, a])
        s, a = ns, na
    return Q

epsilon_values = [0.05, 0.1, 0.2, 0.5]
sarsa_V1 = {}

for eps in epsilon_values:
    np.random.seed(42)
    Q_s = sarsa(env, epsilon=eps)
    V_s = np.zeros(17)
    for _ in range(1000):
        V_new = V_s.copy()
        for s in range(1, 17):
            if s == 16: continue
            acts = env.available(s)
            if acts:
                ba = max(acts, key=lambda a: Q_s[s, a])
                ns, r = env.step(s, ba)
                V_new[s] = r + 0.9 * V_s[ns]
        if np.allclose(V_s, V_new): break
        V_s = V_new
    sarsa_V1[eps] = V_s[1]
    print(f"SARSA ε={eps:.2f}  V(S1) = {V_s[1]:.4f}")

### Task 4 — Algorithm Comparison

In [ ]:
algorithms = ['Q-Learning'] + [f'SARSA\n(ε={e})' for e in epsilon_values]
values     = [V_ql[1]]     + [sarsa_V1[e] for e in epsilon_values]
colors     = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']

fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.bar(algorithms, values, color=colors, width=0.55, edgecolor='black', linewidth=0.8)
for bar, val in zip(bars, values):
    ypos = bar.get_height() + (0.1 if val >= 0 else -0.5)
    ax.text(bar.get_x() + bar.get_width() / 2, ypos,
            f'{val:.2f}', ha='center', va='bottom', fontsize=11, fontweight='bold')
ax.axhline(0, color='black', linewidth=0.8, linestyle='--', alpha=0.5)
ax.set_ylabel('V(State 1)', fontsize=12)
ax.set_title('Algorithm Comparison — V(S1) under Greedy Policy', fontsize=13, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.savefig('results/rl_comparison.png', dpi=150, bbox_inches='tight')
plt.close()
print('Saved results/rl_comparison.png')

Q-Learning and low-ε SARSA (ε=0.05, 0.2) converged to comparable V(S1) values around 0.72. SARSA with ε=0.1 converged to a slightly lower value, reflecting variance from the on-policy stochastic updates. SARSA with ε=0.5 performed worst (V(S1) ≈ −5.26) because half of all actions were random — the agent spent too much time in high-penalty states and never reliably learned the optimal path to S16.

The key distinction: Q-Learning is off-policy (always updates using the max Q of the next state regardless of what it would actually do), while SARSA is on-policy (updates using the action it actually selects, including exploratory ones). At high ε, SARSA's on-policy updates are heavily corrupted by random actions, leading to much worse value estimates.